# Загрузка Pandas и очистка данных

In [1]:
import os
import pandas as pd

In [2]:
main_task_path = os.path.join('data', 'main_task.csv')

In [3]:
df = pd.read_csv(main_task_path)

In [4]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [6]:
# Удалим все столбцы, которые содержат нечисловые данные
df = df.drop(['Restaurant_id', 'City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA'], axis=1)

In [7]:
# Посмотрим на таблицу
df.head()

,Ranking,Rating,Number of Reviews
0,5570.0,3.5,194.0
1,1537.0,4.0,10.0
2,353.0,4.5,688.0
3,3458.0,5.0,3.0
4,621.0,4.0,84.0


In [8]:
# Проверим таблицу на предмет отсутствующих данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Ranking            40000 non-null  float64
 1   Rating             40000 non-null  float64
 2   Number of Reviews  37457 non-null  float64
dtypes: float64(3)
memory usage: 937.6 KB


In [9]:
# Как видно, данные отсутствуют местами только в поле 'Number of Reviews'
# Вычислим средне-арифметическое значение в этом поле
number_of_reviews_mean = df['Number of Reviews'].mean()
number_of_reviews_mean

124.82547988359985

In [10]:
# Заполним пропуски полученным средним значением
df['Number of Reviews'].fillna(number_of_reviews_mean, inplace=True)
# Забегая вперед: значение MAE при заполнении средним значением составило: 0.4276012702380953
# Заполним пропуски нулями
# df['Number of Reviews'].fillna(0, inplace=True)
# Забегая вперед: значение MAE при заполнении нулями составило: 0.42706587858946615
# Вывод: заполнение нулями дает лучшую точность предсказания

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Ranking            40000 non-null  float64
 1   Rating             40000 non-null  float64
 2   Number of Reviews  40000 non-null  float64
dtypes: float64(3)
memory usage: 937.6 KB


In [11]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Rating'], axis = 1)
y = df['Rating']

In [12]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [13]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=101)

# Создаём, обучаем и тестируем модель

In [14]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [15]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [16]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.4281228398809524
